<a href="https://colab.research.google.com/github/nupurpathare/Inventory-Management-System/blob/main/IndianHeritagePreservation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')





MessageError: Error: credential propagation was unsuccessful

In [ ]:
#Install the necessary packages
!pip install inference opencv-python-headless matplotlib
!pip install streamlit pyngrok

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.9/570.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.0/906.0 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 11.4 MB/s eta 0:00:00
   ━━━

In [ ]:
# Write the content to app.py
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
from inference_sdk import InferenceHTTPClient
import cv2
import tempfile
import numpy as np
import matplotlib.pyplot as plt
import json
from io import BytesIO
from PIL import Image
import pandas as pd
from datetime import datetime






#Information about two models
models = {
    "Crack Detection Model 1": {
        "model_id": "crack-detection-x9bz6/1",
        "architecture": "COCOn",
        "training_data": "Crack Images Dataset",
        "accuracy": "87%",
        "initialization": "Fixed Endpoints"
    },
    "Crack Detection Model 2": {
        "model_id": "crack-detection-x9bz6/3",
        "architecture": "COCOn",
        "training_data": " Crack Images Dataset",
        "accuracy": "86%",
        "initialization": "Random"
    }
}

#Function to generate a heatmap image
def generate_heatmap(image, detections):
    heatmap = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)
    for detection in detections:
        x, y, w, h = detection['x'], detection['y'], detection['width'], detection['height']
        confidence = detection['confidence']
        start_point = (int(x - w / 2), int(y - h / 2))
        end_point = (int(x + w / 2), int(y + h / 2))
        cv2.rectangle(heatmap, start_point, end_point, confidence, -1)

    heatmap = np.clip(heatmap, 0, 1)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)
    return overlay

#Function to save the feedback to a csv file named feedback.csv
def save_feedback(feedback_data):
    feedback_file = "feedback.csv"
    new_feedback_df = pd.DataFrame([feedback_data])

    try:
        # Load existing feedback if it exists
        feedback_df = pd.read_csv(feedback_file)
        # Concatenate new feedback with existing feedback
        feedback_df = pd.concat([feedback_df, new_feedback_df], ignore_index=True)
    except FileNotFoundError:
        # If the file does not exist, use the new feedback dataframe
        feedback_df = new_feedback_df

    # Save the updated feedback
    feedback_df.to_csv(feedback_file, index=False)

#Function to preprocess the image i.e convert it to grayscale
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    standardized_image = cv2.equalizeHist(gray_image)  # Standardize the image
    return standardized_image

# Function to perform inference and visualize detections
def process_image(image_file, client, threshold, model_id):

    # Save uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(image_file.read())
        temp_file_path = tmp_file.name

    # Load the image
    image = cv2.imread(temp_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    preprocessed_image_rgb = cv2.cvtColor(preprocessed_image, cv2.COLOR_GRAY2RGB)  # Convert to RGB for display

    # Display the preprocessed image with a button
    if st.button(f"View Preprocessed Image"):
        st.image(preprocessed_image_rgb, caption=f'Preprocessed Image: {image_file.name}', use_column_width=True)

    # Perform inference
    result = client.infer(temp_file_path, model_id=model_id)

    # Parse the result
    detections = result['predictions']

    # Visualize the detections
    for detection in detections:
        # Get the bounding box coordinates and class
        x, y, w, h = detection['x'], detection['y'], detection['width'], detection['height']
        class_name = detection['class']
        confidence = detection['confidence']

        # Draw the bounding box
        if confidence < threshold:
            continue
        start_point = (int(x - w / 2), int(y - h / 2))
        end_point = (int(x + w / 2), int(y + h / 2))
        color = (255, 0, 0)  # Red color for bounding box
        thickness = 2
        image = cv2.rectangle(image, start_point, end_point, color, thickness)

        # Prepare the label with class name and confidence
        label = f"{class_name}: {confidence:.2f}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)

        # Calculate the position to place the label
        label_x = start_point[0]
        label_y = start_point[1] - 10 if start_point[1] - 10 > 10 else start_point[1] + label_size[1] + 10

        # Draw the label on the image
        cv2.putText(image, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    processed_image = Image.fromarray(image)
    # Display the processed image with detections
    st.image(processed_image, caption=f'Processed Image: {image_file.name}', use_column_width=True)

    buffered = BytesIO()
    processed_image.save(buffered, format="JPEG")
    st.download_button(
        label="Download Processed Image",
        data=buffered.getvalue(),
        file_name=f"processed_{image_file.name}",
        mime="image/jpeg"
    )

    heatmap_overlay = generate_heatmap(image, detections)
    heatmap_image = Image.fromarray(heatmap_overlay)
    st.image(heatmap_image, caption='Heatmap Overlay', use_column_width=True)

    buffered = BytesIO()
    heatmap_image.save(buffered, format = "JPEG")
    st.download_button(
      label="Download Heatmap Image",
        data=buffered.getvalue(),
        file_name=f"heatmapped_{image_file.name}",
        mime="image/jpeg"
    )

    # Print the raw result
    st.write("## Inference Result")
    st.write(f"**Time:** {result['time']} seconds")
    st.write(f"**Image Width:** {result['image']['width']}")
    st.write(f"**Image Height:** {result['image']['height']}")

    st.write("### Predictions")
    num_cracks = 0
    confidence_sum = 0
    for i, detection in enumerate(result['predictions'], 1):
        num_cracks += 1
        confidence_sum += detection['confidence']

        st.markdown(f"#### Detection {i}")
        st.write(f"**Class:** {detection['class']}")
        st.write(f"**Confidence:** {detection['confidence']:.2f}")
        st.write(f"**Coordinates (x, y, w, h):** {detection['x']}, {detection['y']}, {detection['width']}, {detection['height']}")
        st.write(f"**Detection ID:** {detection['detection_id']}")
        st.markdown("---")

    # Display detection statistics
    if num_cracks > 0:
        avg_confidence = confidence_sum / num_cracks
    else:
        avg_confidence = 0.0
    st.write("## Detection Statistics")
    st.write(f"**Number of Cracks Detected:** {num_cracks}")
    st.write(f"**Average Confidence Score:** {avg_confidence:.2f}")

# Main Streamlit app
st.title("Crack Detection")
st.write("This app allows you to upload images and detect cracks using a pre-trained model.")
st.write("Upload one or more images to begin.")

st.sidebar.title("Configurations")
selected_model_name = st.sidebar.selectbox("Select Model", list(models.keys()))
threshold = st.sidebar.slider("Detection Threshold", 0.0, 1.0, 0.5, 0.01, help="Adjust the confidence threshold for detections.")
model_id = models[selected_model_name]["model_id"]
st.sidebar.title("Model Information")
selected_model_info = models[selected_model_name]
st.sidebar.write(f"**Architecture:** {selected_model_info['architecture']}")
st.sidebar.write(f"**Training Data:** {selected_model_info['training_data']}")
st.sidebar.write(f"**Accuracy:** {selected_model_info['accuracy']}")
st.sidebar.write(f"**Checkpoint:** {selected_model_info['initialization']}")

uploaded_files = st.file_uploader("Choose images...", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

# Initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="j0MH27jypjyElHGx8xlJ"  # Make sure the API key is correct
)

# Process each uploaded file
if uploaded_files:
    for uploaded_file in uploaded_files:
        with st.spinner(f"Processing {uploaded_file.name}..."):
            try:
                process_image(uploaded_file, CLIENT, threshold, model_id)
            except Exception as e:
                st.error(f"An error occurred while processing the file {uploaded_file.name}: {e}")
    st.success("Processing completed for all uploaded files.")

    st.write("## Provide Feedback")
    for uploaded_file in uploaded_files:
        st.write(f"### Feedback for {uploaded_file.name}")
        rating = st.slider(f"Rate the accuracy of detections for {uploaded_file.name}", 1, 5, 3)
        comments = st.text_area(f"Additional comments for {uploaded_file.name}")

        if st.button(f"Submit Feedback for {uploaded_file.name}"):
            feedback_data = {
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "image_name": uploaded_file.name,
                "model_id": selected_model_info['model_id'],
                "rating": rating,
                "comments": comments
            }
            save_feedback(feedback_data)
            st.success(f"Feedback submitted for {uploaded_file.name}")


    """)

# Run the Streamlit App Locally in Colab
import subprocess
import time

#Kill previously running streamlit application on same port


# Start the Streamlit app
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Wait for a few seconds to ensure the app starts
time.sleep(5)

# Check the logs to ensure the app is running
!ps -aux | grep streamlit

# Use ngrok to Expose the Streamlit App
from pyngrok import ngrok

# Set ngrok authtoken
authtoken = "2iS4eQvgI72rtnVlacoZke68fbF_2VqzVNuTkR6VDiYPhpLv6"  # Replace with your ngrok authtoken. Ngrok token will be different for every email
!ngrok authtoken {authtoken}

# Ensure no other ngrok tunnels are active
ngrok.kill()

# Create a public URL for the Streamlit app
public_url = ngrok.connect(addr='8501', proto='http')
print('Streamlit App can be accessed at:', public_url)


root        1335  0.0  0.0   6484  2400 ?        S    17:46   0:00 grep streamlit
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit App can be accessed at: NgrokTunnel: "https://98dd-34-105-72-85.ngrok-free.app" -> "http://localhost:8501"
